In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load the custom-trained YOLOv5 model
model = YOLO('yolov8n.pt')

# Initialize the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()
else:
    print("Webcam successfully opened!")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to grab frame.")
        break

    # Run detection
    results = model(frame)

    # Annotate the frame with bounding boxes and labels using the plot() method
    frame = results[0].plot()  # This will display the bounding boxes directly on the frame

    # Display the annotated frame
    cv2.imshow("Fruit Detection", frame)

    # Exit when 'ESC' is pressed
    if cv2.waitKey(1) == 27:  # ESC key to exit
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [9]:
import logging
import cv2
from ultralytics import solutions

# Set logging level to WARNING to suppress info-level messages
logging.basicConfig(level=logging.WARNING)
cap = cv2.VideoCapture("vid.mp4")
assert cap.isOpened(), "Error reading video file"

# Video writer
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("speed_management.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# speed region points
#speed_region = [(20, 900), (1800, 700), (1800, 260), (20, 460)]
speed_region = [(20, 400), (1800, 400)]
# Initialize speed estimation object
speedestimator = solutions.SpeedEstimator(
    tracker='bytetrack.yaml',
    show=True,
    model="yolov8s.pt",  # More accurate model
    region=speed_region,
    device="cuda",
    conf=0.5,  # Higher confidence threshold
    iou=0.5,  # Reduce false positives
    track_buffer=30,  # Improve tracking consistency
    line_width=3  # Improve bounding box visibility
)

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        break

    results = speedestimator(im0)
    video_writer.write(results.plot_im)  # write the processed frame.

    if cv2.waitKey(1) == 27:  # ESC key to exit
        break

cap.release()
video_writer.release()
cv2.destroyAllWindows()  # destroy all opened windows


Ultralytics Solutions:  {'region': [(20, 400), (1800, 400)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'show': True, 'model': 'yolov8s.pt', 'device': 'cuda', 'conf': 0.5, 'iou': 0.5, 'track_buffer': 30, 'line_width': 3, 'tracker': 'bytetrack.yaml'}

0: 384x640 5 cars, 1 truck, 4.8ms
Speed: 1.3ms preprocess, 4.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(total_tracks=6)

0: 384x640 5 cars, 1 truck, 4.5ms
Speed: 1.1ms preprocess, 4.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(total_tracks=6)

0: 384x640 5 cars, 1 truck, 4.4ms
Speed: 1.1ms preprocess, 4.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
 Results: SolutionResults(total_tracks=6)

0: 384x640 5 cars, 1 truck, 4.4ms
Speed: 1.0ms preprocess, 4.4ms inference, 1.1ms postprocess per image 